In [ ]:
pip install gns3fy

In [2]:
import gns3fy

In [3]:
server = gns3fy.Gns3Connector("http://localhost:3080")

In [4]:
lab = gns3fy.Project(name="GNS_v0", connector=server)

In [5]:
lab.get()
print(lab)

Project(name='GNS_v0', project_id='f9a30b3e-69d7-4287-b267-f60d3b2b7286', status='opened', path='C:\\Users\\User\\GNS3\\projects\\GNS_v0', filename='GNS_v0.gns3', auto_start=False, auto_close=True, auto_open=False, drawing_grid_size=25, grid_size=75, scene_height=1000, scene_width=2000, show_grid=False, show_interface_labels=False, show_layers=False, snap_to_grid=False, supplier=None, variables=None, zoom=100, stats={'drawings': 0, 'links': 10, 'nodes': 7, 'snapshots': 0}, snapshots=None, drawings=None)


In [6]:
import gns3fy

# Define the server object to establish the connection
gns3_server = gns3fy.Gns3Connector("http://localhost:3080")

# Define the lab you want to load and assign the server connector
lab = gns3fy.Project(name="GNS_v0", connector=gns3_server)

# Retrieve its information and display
lab.get()
print(lab)

# Open the project
lab.open()
lab.status

# Verify the stats
lab.stats

# List the names and status of all the nodes in the project
for node in lab.nodes:
    print(f"Node: {node.name} -- Node Type: {node.node_type} -- Status: {node.status}")

Project(name='GNS_v0', project_id='f9a30b3e-69d7-4287-b267-f60d3b2b7286', status='opened', path='C:\\Users\\User\\GNS3\\projects\\GNS_v0', filename='GNS_v0.gns3', auto_start=False, auto_close=True, auto_open=False, drawing_grid_size=25, grid_size=75, scene_height=1000, scene_width=2000, show_grid=False, show_interface_labels=False, show_layers=False, snap_to_grid=False, supplier=None, variables=None, zoom=100, stats={'drawings': 0, 'links': 10, 'nodes': 7, 'snapshots': 0}, snapshots=None, drawings=None)
Node: R1 -- Node Type: dynamips -- Status: started
Node: R2 -- Node Type: dynamips -- Status: started
Node: R3 -- Node Type: dynamips -- Status: started
Node: R4 -- Node Type: dynamips -- Status: started
Node: R5 -- Node Type: dynamips -- Status: started
Node: R6 -- Node Type: dynamips -- Status: started
Node: R7 -- Node Type: dynamips -- Status: started


In [7]:
import gns3fy
gns3_server = gns3fy.Gns3Connector("http://localhost:3080")
projects = gns3_server.get_projects()
for project in projects:
    print(project["name"])

GNS_v0
NRP_TP_3_BGP
NRP_TP_3_BGP_v1
untitled


In [12]:
import gns3fy

# Connect to the GNS3 server
gns3_server = gns3fy.Gns3Connector("http://localhost:3080")

# Define your project name
project_name = "GNS_v0"
project = gns3fy.Project(name=project_name, connector=gns3_server)
project.get()

# Define network intent: router names, interfaces, and IPv6 addresses
network_intent = {
    "Router1": {
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:1::1/64"
        },
        "ripng_interfaces": ["GigabitEthernet1/0"]
    },
    "Router2": {
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:1::2/64"
        },
        "ripng_interfaces": ["GigabitEthernet1/0"]
    },
    # Add more routers here
}

# Function to push configurations via Telnet
def configure_router(node, config_commands):
    node.console_start()  # Start the console if not already running
    with node.console() as telnet:
        telnet.read_until(b"Router>")
        telnet.write(b"enable\n")
        telnet.read_until(b"Router#")
        telnet.write(b"configure terminal\n")
        
        # Apply each configuration command
        for command in config_commands:
            telnet.write(f"{command}\n".encode())
        
        telnet.write(b"end\n")
        telnet.write(b"write memory\n")
        telnet.read_until(b"Router#")

# Iterate over routers and apply configurations
for node in project.nodes:
    if node.name in network_intent:
        router_config = []
        intent = network_intent[node.name]
        
        # Configure IPv6 addresses
        for interface, ipv6_address in intent["interfaces"].items():
            router_config.append(f"interface {interface}")
            router_config.append(f" ipv6 address {ipv6_address}")
            router_config.append(" no shutdown")
        
        # Enable RIPng
        router_config.append("ipv6 router rip RIPNG")
        for interface in intent["ripng_interfaces"]:
            router_config.append(f" interface {interface}")
            router_config.append(" ipv6 rip RIPNG enable")
        
        print(f"Configuring {node.name}...")
        configure_router(node, router_config)
        print(f"{node.name} configured successfully!")


In [7]:
import requests

response = requests.get("http://localhost:3080/v2/projects")
if response.status_code == 200:
    print("Server response:", response.json())
else:
    print(f"Error: {response.status_code}, Response: {response.text}")

Server response: [{'auto_close': True, 'auto_open': False, 'auto_start': False, 'drawing_grid_size': 25, 'filename': 'GNS_v0.gns3', 'grid_size': 75, 'name': 'GNS_v0', 'path': 'C:\\Users\\User\\GNS3\\projects\\GNS_v0', 'project_id': 'f9a30b3e-69d7-4287-b267-f60d3b2b7286', 'scene_height': 1000, 'scene_width': 2000, 'show_grid': False, 'show_interface_labels': False, 'show_layers': False, 'snap_to_grid': False, 'status': 'opened', 'supplier': None, 'variables': None, 'zoom': 100}, {'auto_close': True, 'auto_open': False, 'auto_start': False, 'drawing_grid_size': 25, 'filename': 'NRP_TP_3_BGP.gns3', 'grid_size': 75, 'name': 'NRP_TP_3_BGP', 'path': 'C:\\Users\\User\\GNS3\\projects\\NRP_TP_3_BGP', 'project_id': 'be9ce004-2fb6-4d9b-b286-be5da3984f5d', 'scene_height': 1000, 'scene_width': 2000, 'show_grid': False, 'show_interface_labels': True, 'show_layers': False, 'snap_to_grid': False, 'status': 'closed', 'supplier': None, 'variables': None, 'zoom': 100}, {'auto_close': False, 'auto_open': 

In [16]:
import gns3fy 

gns3_server = gns3fy.Gns3Connector("http://localhost:3080")
project = gns3fy.Project(name="GNS_v0", connector=gns3_server)
project.get()

for node in project.nodes :
    print(node.ports)

[{'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'FastEthernet0/0', 'port_number': 0, 'short_name': 'f0/0'}, {'adapter_number': 1, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'GigabitEthernet1/0', 'port_number': 0, 'short_name': 'g1/0'}, {'adapter_number': 2, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'GigabitEthernet2/0', 'port_number': 0, 'short_name': 'g2/0'}, {'adapter_number': 3, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'GigabitEthernet3/0', 'port_number': 0, 'short_name': 'g3/0'}]
[{'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'FastEthernet0/0', 'port_number': 0, 'short_name': 'f0/0'}, {'adapter_number': 1, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'GigabitEthernet1/0', 'port_number': 0, 'short_name': 'g1/0'}, {'adapter_numb

In [1]:
import gns3fy

gns3_server = gns3fy.Gns3Connector("http://localhost:3080")
project = gns3fy.Project(name="GNS_v0", connector=gns3_server)
project.get()

# Loop through nodes properly
for node_id in project.nodes:
    node = project.get_node(node_id)  # Fetch the node by its ID
    print(f"Node {node['name']}")
    for interface in node['interfaces']:
        print(f"  Interface {interface['name']} - {interface.get('connected_node', 'No connection')}")


TypeError: 'NoneType' object is not subscriptable

In [18]:
import gns3fy

gns3_server = gns3fy.Gns3Connector("http://localhost:3080")
project = gns3fy.Project(name="GNS_v0", connector=gns3_server)
project.get()

for node in project.nodes:
    print(f"Node {node.name} ({node.node_type})")

    # Loop through ports (interfaces) of the node
    for port in node.ports:
        connected_node = port.get('connected_node')  # Check if this interface is connected
        if connected_node:
            print(f"  {port['name']} is connected to {connected_node}")
        else:
            print(f"  {port['name']} is not connected")

Node R1 (dynamips)
  FastEthernet0/0 is not connected
  GigabitEthernet1/0 is not connected
  GigabitEthernet2/0 is not connected
  GigabitEthernet3/0 is not connected
Node R2 (dynamips)
  FastEthernet0/0 is not connected
  GigabitEthernet1/0 is not connected
  GigabitEthernet2/0 is not connected
  GigabitEthernet3/0 is not connected
Node R3 (dynamips)
  FastEthernet0/0 is not connected
  GigabitEthernet1/0 is not connected
  GigabitEthernet2/0 is not connected
  GigabitEthernet3/0 is not connected
Node R4 (dynamips)
  FastEthernet0/0 is not connected
  GigabitEthernet1/0 is not connected
  GigabitEthernet2/0 is not connected
  GigabitEthernet3/0 is not connected
Node R5 (dynamips)
  FastEthernet0/0 is not connected
  GigabitEthernet1/0 is not connected
  GigabitEthernet2/0 is not connected
  GigabitEthernet3/0 is not connected
Node R6 (dynamips)
  FastEthernet0/0 is not connected
  GigabitEthernet1/0 is not connected
  GigabitEthernet2/0 is not connected
  GigabitEthernet3/0 is not co